# Stock data analysis

In [1]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

# If you get an error running this cell, then please head over to the troubleshooting notebook!

# Connecting to OpenAI

In [2]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


API key found and looks good so far!


In [3]:
openai = OpenAI()

In [4]:
from urllib.parse import urlencode
import datetime

headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

In [5]:
class YahooFinanceWebsite:
    def __init__(self, stock_symbol):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.stock_symbol = stock_symbol.upper()

    def __build_url(self, params):
        base_url = f"https://finance.yahoo.com/quote/{self.stock_symbol}/history/"
        query_string = urlencode(params)
        return f"{base_url}?{query_string}"

    def get_stock_data(self):
        datetime_now = datetime.datetime.now()
        datetime_year_ago = datetime_now - datetime.timedelta(days=365)
        params = {"frequency": "1wk", "period1": datetime_year_ago.timestamp(), "period2": datetime_now.timestamp()}
        url = self.__build_url(params)
        response = requests.get(url, headers=headers)

        soup = BeautifulSoup(response.content, 'html.parser')
        
        title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()

        html_table_data = soup.find("table")

        return title, html_table_data

In [6]:
def build_stock_analysis_prompt(stock_symbol, title, stock_table_data):
    sys_prompt = r"""You are an assistant that analyzes the contents of HTML formated table that contains data on a specific stock.
    The HTML table contains the date, open price, close price, low and highs aggregated for every week over one year timeframe.
    Ignoring text, tags or html attributes that might be navigation related. 
    Respond in Markdown format"""
    
    user_prompt = f"The data provided below in the HTML table format for {stock_symbol} from the Yahoo Finances.\
    Make the explaination easy enough for a newbie to understand. \
    Analyze and Summarize the trends on this stock:\n{stock_table_data}\n\n\
    Also, calculate the total returns in percentage one could have expected over this period."
    
    return [
        {"role": "system", "content": sys_prompt},
        {"role": "user", "content": user_prompt}
    ]

In [7]:
def analyze_stock_trends(stock_symbol):
    stock_data_page = YahooFinanceWebsite(stock_symbol)
    title, stock_table_data = stock_data_page.get_stock_data()
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = build_stock_analysis_prompt(stock_symbol, title, stock_table_data)
    )
    return response.choices[0].message.content

def display_analysis(stock_symbol):
    display(Markdown(analyze_stock_trends(stock_symbol)))

In [8]:
display_analysis("GOOG")

## GOOG Stock Trend Analysis

### Overview of Weekly Data
The provided data covers stock prices for Alphabet Inc. (GOOG) on a weekly basis, showing the open, high, low, close prices, adjusted close prices, and trading volume for each week. 

### Key Price Metrics
1. **Open**: Price at which the stock opened at the start of the trading week.
2. **High**: The highest price reached during the week.
3. **Low**: The lowest price reached during the week.
4. **Close**: Price at which the stock closed at the end of the trading week.
5. **Adjusted Close**: The closing price adjusted for dividends and stock splits.

### Price Trend Summary
- **April 2024 - April 2025**: Stock prices for GOOG primarily ranged between **$143.39** (April 7, 2025) and **$202.21** (Feb 3, 2025). 
- **Price Fluctuations**: 
  - The stock shows volatility, with significant price movements up and down.
  - The stock reached its peak during the week of February 3, 2025, but saw declines through March and into April, indicating some recent bearish sentiment.

### Observations from Weekly Prices
- **Recent Decline**: In the last weeks, particularly from March 24, 2025 to the latest data in April 2025, the stock price decreased, indicating recent market pressures.
- **Trading Volume**: The trading volume also exhibited substantial fluctuations with peaks correlating with price changes. Notably high volume in some weeks suggests increased interest or reaction to news related to the company or market conditions.

### Percentage Return Calculation
To calculate total returns over the observed period from the start of the year to the last provided date:

1. **Initial Price (Jan 6, 2025)**: $195.15
2. **Final Price (Apr 15, 2025)**: $158.68

The formula for percentage return is:
\[
\text{Percentage Return} = \left( \frac{\text{Final Price} - \text{Initial Price}}{\text{Initial Price}} \right) \times 100
\]

Calculating:
\[
\text{Percentage Return} = \left( \frac{158.68 - 195.15}{195.15} \right) \times 100 = \left( \frac{-36.47}{195.15} \right) \times 100 \approx -18.67\%
\]

### Conclusion
- The analysis indicates that GOOG has experienced a **decline of approximately 18.67%** over the observed period, reflecting broader market trends or company-specific challenges.
- Investors should be cautious and keep an eye on upcoming earnings reports or news which might influence stock performance in the near future.

In [9]:
display_analysis("PFE")

## Analysis of PFE Stock Trends

### Overview of Stock Data

The data shows historical weekly prices for Pfizer Inc. (PFE) over a timeframe leading up to April 15, 2025. Key metrics include:

- **Open Price:** The price at which the stock opened at the beginning of the trading week.
- **High Price:** The highest price reached during the week.
- **Low Price:** The lowest price reached during the week.
- **Close Price:** The price at which the stock settled at the end of the trading week.
- **Adj Close:** The adjusted closing price accounts for stock splits and dividends.
- **Volume:** The number of shares traded during the week.

### Key Trends Observed

1. **Overall Price Movement:**
   - The stock opened at **22.17** on **April 15, 2025** and closed at **22.44** on the same date.
   - In the weeks preceding, Pfizer's stock experienced fluctuations, ending higher than it started in several instances but also showing notable declines. 

2. **Recent Performance:**
   - The highest observed close was **26.73** on **March 3, 2025**.
   - Notable drops occurred in late March and early April, seeing prices fall from as high as **26.28** to **22.12**.
   - The stock had significant volatility, with a decline to approximately **21.91** from a high in March.

3. **Volume Trends:**
   - Some weeks, especially around the higher price points, show substantial trading volume (e.g. **493,533,700** shares on **March 17, 2025**).
   - Lower trading volumes typically occurred during less volatile periods.

4. **Dividend Reports:**
   - Dividends were reported at **0.43** on **January 24, 2025**, and **0.42** on **November 8, 2024**, among others. This indicates that the stock pays dividends, which can be a valuable component of total returns for investors.

### Total Return Calculation

To calculate the total return over the period, we will use the **initial and final adjusted close prices** to find the percentage return:

- **Adjustment for Low and High Returns:**
   - **Initial Price (Adj Close on April 15, 2024)**: **26.00**
   - **Final Price (Adj Close on April 15, 2025)**: **22.44**
  
Using the formula for total return:

\[
\text{Total Return (\%)} = \left( \frac{\text{Final Price} - \text{Initial Price}}{\text{Initial Price}} \right) \times 100
\]

Substituting the values:

\[
= \left( \frac{22.44 - 26.00}{26.00} \right) \times 100 \\
= \left( \frac{-3.56}{26.00} \right) \times 100 \\
\approx -13.69\%
\]

### Summary

1. **Price Fluctuation:** The stock has been volatile, with highs around **26** and lows dropping under **22**.
2. **Negative Return:** An investor who held PFE from April 2024 to April 2025 would have experienced a total return of approximately **-13.69%**, reflecting a loss.
3. **Dividend Benefits:** Despite the drop in price, the dividends can offer some offset to losses for stockholders, providing a cash return.

As a new investor, it's essential to remember that stock prices can be highly volatile and affected by various factors, including market sentiment, earnings reports, and broader economic conditions. Always assess both price trends and underlying performance for informed investment decisions.

In [10]:
display_analysis("AAPL")

## AAPL Stock Analysis (April 2024 - April 2025)

### Key Data Overview
The provided data shows Apple Inc. (AAPL) stock performance over a year, summarized in a weekly format. The table includes:

- **Date**: The date of trading.
- **Open**: The stock price at the beginning of the week.
- **High**: The highest price reached during that week.
- **Low**: The lowest price during the week.
- **Close**: The stock price at the end of the week.
- **Adj Close**: Adjusted closing price, considering dividends and stock splits.
- **Volume**: The number of shares traded over the week.

### Trends and Observations

1. **Price Fluctuations**:
   - The stock price began at **$175.36** at the start of the week ending April 15, 2024, and ended at **$202.14** by the week ending April 15, 2025.
   - The highest recorded price in this interval was **$254.77** (week ending December 23, 2024).
   - The lowest recorded price was **$164.08** (week ending April 15, 2024).
   - Overall, the stock showed significant price fluctuations—from a low of **$164.08** to a high of **$254.77**.

2. **General Trend**:
   - There is an overall upward trend in the stock price, particularly observed towards the end of the analyzed period.
   - Early 2025 shows recovery from dips seen in late 2024, indicating a rebound in investor sentiment.

3. **Monthly Overview**:
   - **January 2025**: Prices fluctuated between **$222.54** and **$236.00**.
   - **February 2025**: A notable increase, with prices often hovering above **$240**.
   - **March 2025**: A peak was reached with prices hitting above **$225** before a drop back down.

4. **Volume Trends**:
   - The highest trading volumes were observed in early 2025, with notably high trading days (e.g., **675,037,600** shares traded).
   - Higher trading volumes often coincide with market volatility, suggesting increased investor activity during these periods.

5. **Dividends**:
   - Regular dividends were issued throughout the year, indicating a commitment to returning value to shareholders.

### Total Return Calculation
To calculate the total return percentage over the one-year period:

1. **Begin Price (April 15, 2024)**: **$175.36**
2. **End Price (April 15, 2025)**: **$202.14**
3. **Return Calculation**:
   \[
   \text{Total Return} = \left(\frac{\text{End Price} - \text{Start Price}}{\text{Start Price}}\right) \times 100
   \]

   Plugging in the numbers:
   \[
   \text{Total Return} = \left(\frac{202.14 - 175.36}{175.36}\right) \times 100 \approx 15.27\%
   \]

### Conclusion
- Over the analyzed year, AAPL experienced notable price fluctuations but ended significantly higher than it started, marking a **≈ 15.27%** increase in value.
- This performance, alongside the consistent dividend payments, signifies a potentially fruitful investment for shareholders within this timeframe. 

Investing in stocks always involves risk, and trends may vary based on broader market conditions and company performance. Hence, it is advisable to conduct thorough research or consult a financial advisor before making investment decisions.